In [ ]:
import pandas as pd
#!pip install datasets
from datasets import Dataset
from google.colab import files
from transformers import TrainingArguments, Trainer
import numpy as np
from sklearn.metrics import accuracy_score, f1_score
from transformers import AutoModelForSequenceClassification
#uploaded = files.upload()


In [ ]:
df = pd.read_pickle("merged_training.pkl")

In [ ]:
# Remove duplicate texts
duplicates = df['text'].duplicated(keep=False).sum()
print(f"Znaleziono {duplicates} wystąpień powtarzających się wartości w kolumnie 'text'.")
df = df[~df['text'].duplicated(keep=False)]
print(f"Liczba wierszy po usunięciu wszystkich duplikatów: {len(df)}")

# Undersampling to balance classes
counts = df['emotions'].value_counts()
min_count = counts.min()
df = df.groupby('emotions').apply(lambda x: x.sample(n=min_count, random_state=42)).reset_index(drop=True)

# Convert label to category index
df['labels'] = df['emotions'].astype('category').cat.codes

# Save label map for inference
label_map = dict(enumerate(df['emotions'].astype('category').cat.categories))

# Convert to Hugging Face dataset
dataset = Dataset.from_pandas(df[["text", "labels"]])
dataset = dataset.train_test_split(test_size=0.2, seed=42)



In [ ]:
# Tokenization
from transformers import AutoTokenizer

model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize_function(example):
    return tokenizer(example["text"], padding="max_length", truncation=True)

tokenized_dataset = dataset.map(tokenize_function, batched=True)

# Load model
num_labels = len(df['labels'].unique())
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)

# Metrics
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=1)
    return {
        "accuracy": accuracy_score(labels, preds),
        "f1": f1_score(labels, preds, average='weighted')
    }


In [ ]:
# Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

# Train model
trainer.train()



In [ ]:
# Save the model
trainer.save_model("emotion-classifier")

# Evaluate on the test set
print("Ewaluacja na zbiorze testowym:")
metrics = trainer.evaluate(tokenized_dataset["test"])
print(metrics)

